# Imports

In [160]:
import pandas as pd
import pickle
import numpy as np
from sklearn.externals import joblib
from sklearn import *
from sklearn import cluster
from sklearn.metrics import mean_squared_error,mean_absolute_error
from math import sqrt
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.feature_selection import RFE

## Load clean loan csv 

In [161]:
loan_df = pd.read_csv('/Users/madhu/Desktop/ADS Project/Part-1/Loan.csv')

In [162]:
loan_df.columns

Index(['id', 'debt_settlement_flag', 'application_type', 'fico_range_low',
       'fico_range_high', 'emp_length', 'dti', 'annual_inc', 'grade',
       'sub_grade', 'int_rate', 'loan_amnt', 'issue_d', 'purpose', 'State',
       'home_ownership', 'zip_code', 'policy_code', 'term', 'Year', 'Month',
       'fico', 'approval'],
      dtype='object')

In [163]:
loan_df.dtypes

id                        int64
debt_settlement_flag     object
application_type         object
fico_range_low          float64
fico_range_high         float64
emp_length                int64
dti                     float64
annual_inc              float64
grade                    object
sub_grade                object
int_rate                float64
loan_amnt               float64
issue_d                  object
purpose                  object
State                    object
home_ownership           object
zip_code                  int64
policy_code               int64
term                      int64
Year                      int64
Month                     int64
fico                    float64
approval                  int64
dtype: object

In [164]:
loan_df['policy_code'] = loan_df['policy_code'].astype(int)

# Remove General Columns Not Needed

In [167]:
feature_df = loan_df[['application_type','emp_length', 'dti', 'annual_inc',
       'int_rate', 'loan_amnt', 'purpose', 'State', 'home_ownership', 'term', 'fico', 'approval']]

In [168]:
feature_df.columns

Index(['application_type', 'emp_length', 'dti', 'annual_inc', 'int_rate',
       'loan_amnt', 'purpose', 'State', 'home_ownership', 'term', 'fico',
       'approval'],
      dtype='object')

# Dealing with catagorical variables

In [169]:
home_ownerships = pd.get_dummies(feature_df['home_ownership'], prefix='house')
feature_df = feature_df.join(home_ownerships)
feature_df.drop('home_ownership', axis=1, inplace=True)

app_type = pd.get_dummies(feature_df['application_type'], prefix='appType')
feature_df = feature_df.join(app_type)
feature_df.drop('application_type', axis=1, inplace=True)

adr_states = pd.get_dummies(feature_df['State'], prefix='s')
feature_df = feature_df.join(adr_states)
feature_df.drop('State', axis=1, inplace=True)

#purpose   
purpose = pd.get_dummies(feature_df['purpose'], prefix='purpose')
feature_df = feature_df.join(purpose)
feature_df.drop('purpose', axis=1, inplace=True)

# No Clustering

In [145]:
zero_cluster_df = feature_df

In [146]:
X_train, X_test, y_train, y_test = train_test_split(zero_cluster_df.ix[:, zero_cluster_df.columns != 'int_rate'], 
                                            zero_cluster_df['int_rate'], 
                                            test_size=0.30)
lm=linear_model.LinearRegression()
lm.fit(X_train,y_train)
y_train_predicted = lm.predict(X_train)
lm.fit(X_test,y_test)
y_test_predicted = lm.predict(X_test)
#finished square footage of the property, square footage of lot, tax amounts, number of bedrooms, and number of units
#MAE, RMS, MAPE
rms = sqrt(mean_squared_error(y_train, y_train_predicted))    
mae_train = mean_absolute_error(y_train, y_train_predicted) 
mape_train = np.mean(np.abs((y_train - y_train_predicted) / y_train)) * 100
print("Train")
print(rms)
print(mae_train)
print(mape_train)
#finished square footage of the property, square footage of lot, tax amounts, number of bedrooms, and number of units
#MAE, RMS, MAPE
rms = sqrt(mean_squared_error(y_test, y_test_predicted))    
mae_test = mean_absolute_error(y_test, y_test_predicted)
mape_test = np.mean(np.abs((y_test - y_test_predicted) / y_test)) * 100
print("Test")
print(rms)
print(mae_test)
print(mape_test)

C:\Users\madhu\Anaconda3\lib\site-packages\ipykernel_launcher.py:1: DeprecationWarning: 
.ix is deprecated. Please use
.loc for label based indexing or
.iloc for positional indexing

See the documentation here:
http://pandas.pydata.org/pandas-docs/stable/indexing.html#deprecate_ix
  """Entry point for launching an IPython kernel.


Train
3.712511343697658
2.84995954278
23.633236194909287
Test
3.699412729925242
2.84114015302
23.562978838485247


# Manual Clustering

In [150]:
manual_df = feature_df
def clustering(fico):
    cluster_name = ''
    if fico>790.0:
        cluster_name ='K1'
    elif ((fico <= 790.0) & (fico>750.0)):
        cluster_name = 'K2'
    elif ((fico <=750.0) & (fico>700.0)):
        cluster_name ='K3'
    elif ((fico <=700.0) & (fico>660.0)):
        cluster_name= 'K4'
    return cluster_name
manual_df['cluster'] = manual_df['fico'].astype(float).map(lambda x: clustering(x))

In [151]:
manual_df.head()

,emp_length,dti,annual_inc,int_rate,loan_amnt,term,fico,approval,house_ANY,house_MORTGAGE,...,purpose_house,purpose_major_purchase,purpose_medical,purpose_moving,purpose_other,purpose_renewable_energy,purpose_small_business,purpose_vacation,purpose_wedding,cluster
0,10,27.65,24000.0,10.65,5000.0,36,737.0,1,0,0,...,0,0,0,0,0,0,0,0,0,K3
1,1,1.00,30000.0,15.27,2500.0,60,742.0,1,0,0,...,0,0,0,0,0,0,0,0,0,K3
2,10,8.72,12252.0,15.96,2400.0,36,737.0,1,0,0,...,0,0,0,0,0,0,1,0,0,K3
3,10,20.00,49200.0,13.49,10000.0,36,692.0,1,0,0,...,0,0,0,0,1,0,0,0,0,K4
4,1,17.94,80000.0,12.69,3000.0,60,697.0,1,0,0,...,0,0,0,0,1,0,0,0,0,K4


In [152]:
#Seperate into diff dataframes
manual_c1 = manual_df[manual_df['cluster']=='K1']
manual_c1.drop('cluster',axis=1,inplace=True)
manual_c2 = manual_df[manual_df['cluster']=='K2']
manual_c2.drop('cluster',axis=1,inplace=True)
manual_c3 = manual_df[manual_df['cluster']=='K3']
manual_c3.drop('cluster',axis=1,inplace=True)
manual_c4 = manual_df[manual_df['cluster']=='K4']
manual_c4.drop('cluster',axis=1,inplace=True)

C:\Users\madhu\Anaconda3\lib\site-packages\ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  This is separate from the ipykernel package so we can avoid doing imports until
C:\Users\madhu\Anaconda3\lib\site-packages\ipykernel_launcher.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """
C:\Users\madhu\Anaconda3\lib\site-packages\ipykernel_launcher.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  import sys
C:\Users\madhu\Anaconda3\lib\site-packages\ipykernel_

In [153]:
#Manual 1
X_train, X_test, y_train, y_test = train_test_split(manual_c1.ix[:, manual_c1.columns != 'int_rate'], 
                                            manual_c1['int_rate'], 
                                            test_size=0.30)
lm=linear_model.LinearRegression()
lm.fit(X_train,y_train)
y_train_predicted = lm.predict(X_train)
lm.fit(X_test,y_test)
y_test_predicted = lm.predict(X_test)
#finished square footage of the property, square footage of lot, tax amounts, number of bedrooms, and number of units
#MAE, RMS, MAPE
rms = sqrt(mean_squared_error(y_train, y_train_predicted))    
mae_train = mean_absolute_error(y_train, y_train_predicted) 
mape_train = np.mean(np.abs((y_train - y_train_predicted) / y_train)) * 100
print("Train")
print(rms)
print(mae_train)
print(mape_train)
#finished square footage of the property, square footage of lot, tax amounts, number of bedrooms, and number of units
#MAE, RMS, MAPE
rms = sqrt(mean_squared_error(y_test, y_test_predicted))    
mae_test = mean_absolute_error(y_test, y_test_predicted)
mape_test = np.mean(np.abs((y_test - y_test_predicted) / y_test)) * 100
print("Test")
print(rms)
print(mae_test)
print(mape_test)

Train
2.5296695758527212
1.75937376646
20.985107843446343
Test
2.531795436852632
1.75843922297
21.02752283146563


C:\Users\madhu\Anaconda3\lib\site-packages\ipykernel_launcher.py:2: DeprecationWarning: 
.ix is deprecated. Please use
.loc for label based indexing or
.iloc for positional indexing

See the documentation here:
http://pandas.pydata.org/pandas-docs/stable/indexing.html#deprecate_ix
  


In [154]:
#manual 2
X_train, X_test, y_train, y_test = train_test_split(manual_c2.ix[:, manual_c2.columns != 'int_rate'], 
                                            manual_c2['int_rate'], 
                                            test_size=0.30)
lm=linear_model.LinearRegression()
lm.fit(X_train,y_train)
y_train_predicted = lm.predict(X_train)
lm.fit(X_test,y_test)
y_test_predicted = lm.predict(X_test)
#finished square footage of the property, square footage of lot, tax amounts, number of bedrooms, and number of units
#MAE, RMS, MAPE
rms = sqrt(mean_squared_error(y_train, y_train_predicted))    
mae_train = mean_absolute_error(y_train, y_train_predicted) 
mape_train = np.mean(np.abs((y_train - y_train_predicted) / y_train)) * 100
print("Train")
print(rms)
print(mae_train)
print(mape_train)
#finished square footage of the property, square footage of lot, tax amounts, number of bedrooms, and number of units
#MAE, RMS, MAPE
rms = sqrt(mean_squared_error(y_test, y_test_predicted))    
mae_test = mean_absolute_error(y_test, y_test_predicted)
mape_test = np.mean(np.abs((y_test - y_test_predicted) / y_test)) * 100
print("Test")
print(rms)
print(mae_test)
print(mape_test)

C:\Users\madhu\Anaconda3\lib\site-packages\ipykernel_launcher.py:2: DeprecationWarning: 
.ix is deprecated. Please use
.loc for label based indexing or
.iloc for positional indexing

See the documentation here:
http://pandas.pydata.org/pandas-docs/stable/indexing.html#deprecate_ix
  


Train
3.0177963030542627
2.17445165602
23.95867111757733
Test
3.0244965543836257
2.18892584253
24.164364029994683


In [155]:
#Manual 3
X_train, X_test, y_train, y_test = train_test_split(manual_c3.ix[:, manual_c3.columns != 'int_rate'], 
                                            manual_c3['int_rate'], 
                                            test_size=0.30)
lm=linear_model.LinearRegression()
lm.fit(X_train,y_train)
y_train_predicted = lm.predict(X_train)
lm.fit(X_test,y_test)
y_test_predicted = lm.predict(X_test)
#finished square footage of the property, square footage of lot, tax amounts, number of bedrooms, and number of units
#MAE, RMS, MAPE
rms = sqrt(mean_squared_error(y_train, y_train_predicted))    
mae_train = mean_absolute_error(y_train, y_train_predicted) 
mape_train = np.mean(np.abs((y_train - y_train_predicted) / y_train)) * 100
print("Train")
print(rms)
print(mae_train)
print(mape_train)
#finished square footage of the property, square footage of lot, tax amounts, number of bedrooms, and number of units
#MAE, RMS, MAPE
rms = sqrt(mean_squared_error(y_test, y_test_predicted))    
mae_test = mean_absolute_error(y_test, y_test_predicted)
mape_test = np.mean(np.abs((y_test - y_test_predicted) / y_test)) * 100
print("Test")
print(rms)
print(mae_test)
print(mape_test)

C:\Users\madhu\Anaconda3\lib\site-packages\ipykernel_launcher.py:2: DeprecationWarning: 
.ix is deprecated. Please use
.loc for label based indexing or
.iloc for positional indexing

See the documentation here:
http://pandas.pydata.org/pandas-docs/stable/indexing.html#deprecate_ix
  


Train
3.526840934941305
2.68736780253
24.925333569988457
Test
3.5311655850265855
2.6947859014
24.97218271841321


In [156]:
#Manual 4
X_train, X_test, y_train, y_test = train_test_split(manual_c4.ix[:, manual_c4.columns != 'int_rate'], 
                                            manual_c4['int_rate'], 
                                            test_size=0.30)
lm=linear_model.LinearRegression()
lm.fit(X_train,y_train)
y_train_predicted = lm.predict(X_train)
lm.fit(X_test,y_test)
y_test_predicted = lm.predict(X_test)
#finished square footage of the property, square footage of lot, tax amounts, number of bedrooms, and number of units
#MAE, RMS, MAPE
rms = sqrt(mean_squared_error(y_train, y_train_predicted))    
mae_train = mean_absolute_error(y_train, y_train_predicted) 
mape_train = np.mean(np.abs((y_train - y_train_predicted) / y_train)) * 100
print("Train")
print(rms)
print(mae_train)
print(mape_train)
#finished square footage of the property, square footage of lot, tax amounts, number of bedrooms, and number of units
#MAE, RMS, MAPE
rms = sqrt(mean_squared_error(y_test, y_test_predicted))    
mae_test = mean_absolute_error(y_test, y_test_predicted)
mape_test = np.mean(np.abs((y_test - y_test_predicted) / y_test)) * 100
print("Test")
print(rms)
print(mae_test)
print(mape_test)

C:\Users\madhu\Anaconda3\lib\site-packages\ipykernel_launcher.py:2: DeprecationWarning: 
.ix is deprecated. Please use
.loc for label based indexing or
.iloc for positional indexing

See the documentation here:
http://pandas.pydata.org/pandas-docs/stable/indexing.html#deprecate_ix
  


Train
3.8483258925447466
2.98094161718
22.390461040296454
Test
3.839516402430663
2.97350657605
22.30874246406125


# K-Means

In [171]:
k_cluster = joblib.load(open('../kmeanCluster.pkl', 'rb'))
#Create new column in load_df for the k-means clustering 
k_mean_feature_df = feature_df
k_mean_feature_df['kMean'] = k_cluster.labels_

In [172]:
kmean_0 = k_mean_feature_df[k_mean_feature_df['kMean']==0]
kmean_0.drop('kMean', axis=1, inplace=True)
kmean_1 = k_mean_feature_df[k_mean_feature_df['kMean']==1]
kmean_1.drop('kMean', axis=1, inplace=True)
kmean_2 = k_mean_feature_df[k_mean_feature_df['kMean']==2]
kmean_2.drop('kMean', axis=1, inplace=True)
kmean_3 = k_mean_feature_df[k_mean_feature_df['kMean']==3]
kmean_3.drop('kMean', axis=1, inplace=True)
kmean_4 = k_mean_feature_df[k_mean_feature_df['kMean']==4]
kmean_4.drop('kMean', axis=1, inplace=True)
kmean_5 = k_mean_feature_df[k_mean_feature_df['kMean']==5]
kmean_5.drop('kMean', axis=1, inplace=True)
kmean_6 = k_mean_feature_df[k_mean_feature_df['kMean']==6]
kmean_6.drop('kMean', axis=1, inplace=True)
kmean_7 = k_mean_feature_df[k_mean_feature_df['kMean']==7]
kmean_7.drop('kMean', axis=1, inplace=True)


C:\Users\madhu\Anaconda3\lib\site-packages\ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  
C:\Users\madhu\Anaconda3\lib\site-packages\ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  after removing the cwd from sys.path.
C:\Users\madhu\Anaconda3\lib\site-packages\ipykernel_launcher.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  
C:\Users\madhu\Anaconda3\lib\site-packages\ipykernel_launcher.py:8: SettingWithCopyWarning: 
A value is tryi

In [173]:
#K-mean_0
X_train, X_test, y_train, y_test = train_test_split(kmean_0.ix[:, kmean_0.columns != 'int_rate'], 
                                            kmean_0['int_rate'], 
                                            test_size=0.30)
lm=linear_model.LinearRegression()
lm.fit(X_train,y_train)
y_train_predicted = lm.predict(X_train)
lm.fit(X_test,y_test)
y_test_predicted = lm.predict(X_test)
#finished square footage of the property, square footage of lot, tax amounts, number of bedrooms, and number of units
#MAE, RMS, MAPE
rms = sqrt(mean_squared_error(y_train, y_train_predicted))    
mae_train = mean_absolute_error(y_train, y_train_predicted) 
mape_train = np.mean(np.abs((y_train - y_train_predicted) / y_train)) * 100
print("Train")
print(rms)
print(mae_train)
print(mape_train)
#finished square footage of the property, square footage of lot, tax amounts, number of bedrooms, and number of units
#MAE, RMS, MAPE
rms = sqrt(mean_squared_error(y_test, y_test_predicted))    
mae_test = mean_absolute_error(y_test, y_test_predicted)
mape_test = np.mean(np.abs((y_test - y_test_predicted) / y_test)) * 100
print("Test")
print(rms)
print(mae_test)
print(mape_test)

C:\Users\madhu\Anaconda3\lib\site-packages\ipykernel_launcher.py:2: DeprecationWarning: 
.ix is deprecated. Please use
.loc for label based indexing or
.iloc for positional indexing

See the documentation here:
http://pandas.pydata.org/pandas-docs/stable/indexing.html#deprecate_ix
  


Train
3.4395804117780338
2.63133469731
23.054308119310033
Test
3.436692691662597
2.6357190358
23.148686122285792


In [174]:
#K-mean_1
X_train, X_test, y_train, y_test = train_test_split(kmean_1.ix[:, kmean_1.columns != 'int_rate'], 
                                            kmean_1['int_rate'], 
                                            test_size=0.30)
lm=linear_model.LinearRegression()
lm.fit(X_train,y_train)
y_train_predicted = lm.predict(X_train)
lm.fit(X_test,y_test)
y_test_predicted = lm.predict(X_test)
#finished square footage of the property, square footage of lot, tax amounts, number of bedrooms, and number of units
#MAE, RMS, MAPE
rms = sqrt(mean_squared_error(y_train, y_train_predicted))    
mae_train = mean_absolute_error(y_train, y_train_predicted) 
mape_train = np.mean(np.abs((y_train - y_train_predicted) / y_train)) * 100
print("Train")
print(rms)
print(mae_train)
print(mape_train)
#finished square footage of the property, square footage of lot, tax amounts, number of bedrooms, and number of units
#MAE, RMS, MAPE
rms = sqrt(mean_squared_error(y_test, y_test_predicted))    
mae_test = mean_absolute_error(y_test, y_test_predicted)
mape_test = np.mean(np.abs((y_test - y_test_predicted) / y_test)) * 100
print("Test")
print(rms)
print(mae_test)
print(mape_test)

C:\Users\madhu\Anaconda3\lib\site-packages\ipykernel_launcher.py:2: DeprecationWarning: 
.ix is deprecated. Please use
.loc for label based indexing or
.iloc for positional indexing

See the documentation here:
http://pandas.pydata.org/pandas-docs/stable/indexing.html#deprecate_ix
  


Train
3.9437099735682213
3.06277096772
22.074073506756157
Test
3.9430832931862123
3.06790018829
22.101404972290855


In [175]:
#K-mean_2
X_train, X_test, y_train, y_test = train_test_split(kmean_2.ix[:, kmean_2.columns != 'int_rate'], 
                                            kmean_2['int_rate'], 
                                            test_size=0.30)
lm=linear_model.LinearRegression()
lm.fit(X_train,y_train)
y_train_predicted = lm.predict(X_train)
lm.fit(X_test,y_test)
y_test_predicted = lm.predict(X_test)
#finished square footage of the property, square footage of lot, tax amounts, number of bedrooms, and number of units
#MAE, RMS, MAPE
rms = sqrt(mean_squared_error(y_train, y_train_predicted))    
mae_train = mean_absolute_error(y_train, y_train_predicted) 
mape_train = np.mean(np.abs((y_train - y_train_predicted) / y_train)) * 100
print("Train")
print(rms)
print(mae_train)
print(mape_train)
#finished square footage of the property, square footage of lot, tax amounts, number of bedrooms, and number of units
#MAE, RMS, MAPE
rms = sqrt(mean_squared_error(y_test, y_test_predicted))    
mae_test = mean_absolute_error(y_test, y_test_predicted)
mape_test = np.mean(np.abs((y_test - y_test_predicted) / y_test)) * 100
print("Test")
print(rms)
print(mae_test)
print(mape_test)

C:\Users\madhu\Anaconda3\lib\site-packages\ipykernel_launcher.py:2: DeprecationWarning: 
.ix is deprecated. Please use
.loc for label based indexing or
.iloc for positional indexing

See the documentation here:
http://pandas.pydata.org/pandas-docs/stable/indexing.html#deprecate_ix
  


Train
3.5192828354459897
2.70123915385
24.437009789192825
Test
3.520564928592878
2.69291094861
24.37478515799758


In [176]:
#K-mean_3
X_train, X_test, y_train, y_test = train_test_split(kmean_3.ix[:, kmean_3.columns != 'int_rate'], 
                                            kmean_3['int_rate'], 
                                            test_size=0.30)
lm=linear_model.LinearRegression()
lm.fit(X_train,y_train)
y_train_predicted = lm.predict(X_train)
lm.fit(X_test,y_test)
y_test_predicted = lm.predict(X_test)
#finished square footage of the property, square footage of lot, tax amounts, number of bedrooms, and number of units
#MAE, RMS, MAPE
rms = sqrt(mean_squared_error(y_train, y_train_predicted))    
mae_train = mean_absolute_error(y_train, y_train_predicted) 
mape_train = np.mean(np.abs((y_train - y_train_predicted) / y_train)) * 100
print("Train")
print(rms)
print(mae_train)
print(mape_train)
#finished square footage of the property, square footage of lot, tax amounts, number of bedrooms, and number of units
#MAE, RMS, MAPE
rms = sqrt(mean_squared_error(y_test, y_test_predicted))    
mae_test = mean_absolute_error(y_test, y_test_predicted)
mape_test = np.mean(np.abs((y_test - y_test_predicted) / y_test)) * 100
print("Test")
print(rms)
print(mae_test)
print(mape_test)

C:\Users\madhu\Anaconda3\lib\site-packages\ipykernel_launcher.py:2: DeprecationWarning: 
.ix is deprecated. Please use
.loc for label based indexing or
.iloc for positional indexing

See the documentation here:
http://pandas.pydata.org/pandas-docs/stable/indexing.html#deprecate_ix
  


Train
3.5713743802550444
2.73566446554
22.690182006212158
Test
3.589818163628982
2.74893879229
22.77619272088529


In [177]:
#K-mean_4
X_train, X_test, y_train, y_test = train_test_split(kmean_4.ix[:, kmean_4.columns != 'int_rate'], 
                                            kmean_4['int_rate'], 
                                            test_size=0.30)
lm=linear_model.LinearRegression()
lm.fit(X_train,y_train)
y_train_predicted = lm.predict(X_train)
lm.fit(X_test,y_test)
y_test_predicted = lm.predict(X_test)
#finished square footage of the property, square footage of lot, tax amounts, number of bedrooms, and number of units
#MAE, RMS, MAPE
rms = sqrt(mean_squared_error(y_train, y_train_predicted))    
mae_train = mean_absolute_error(y_train, y_train_predicted) 
mape_train = np.mean(np.abs((y_train - y_train_predicted) / y_train)) * 100
print("Train")
print(rms)
print(mae_train)
print(mape_train)
#finished square footage of the property, square footage of lot, tax amounts, number of bedrooms, and number of units
#MAE, RMS, MAPE
rms = sqrt(mean_squared_error(y_test, y_test_predicted))    
mae_test = mean_absolute_error(y_test, y_test_predicted)
mape_test = np.mean(np.abs((y_test - y_test_predicted) / y_test)) * 100
print("Test")
print(rms)
print(mae_test)
print(mape_test)

Train
4.3241626529673285
3.29343073359
22.145839177219973
Test
4.395156814949426
3.38682850981
22.794969487569674


C:\Users\madhu\Anaconda3\lib\site-packages\ipykernel_launcher.py:2: DeprecationWarning: 
.ix is deprecated. Please use
.loc for label based indexing or
.iloc for positional indexing

See the documentation here:
http://pandas.pydata.org/pandas-docs/stable/indexing.html#deprecate_ix
  


In [178]:
#K-mean_5
X_train, X_test, y_train, y_test = train_test_split(kmean_5.ix[:, kmean_5.columns != 'int_rate'], 
                                            kmean_5['int_rate'], 
                                            test_size=0.30)
lm=linear_model.LinearRegression()
lm.fit(X_train,y_train)
y_train_predicted = lm.predict(X_train)
lm.fit(X_test,y_test)
y_test_predicted = lm.predict(X_test)
#finished square footage of the property, square footage of lot, tax amounts, number of bedrooms, and number of units
#MAE, RMS, MAPE
rms = sqrt(mean_squared_error(y_train, y_train_predicted))    
mae_train = mean_absolute_error(y_train, y_train_predicted) 
mape_train = np.mean(np.abs((y_train - y_train_predicted) / y_train)) * 100
print("Train")
print(rms)
print(mae_train)
print(mape_train)
#finished square footage of the property, square footage of lot, tax amounts, number of bedrooms, and number of units
#MAE, RMS, MAPE
rms = sqrt(mean_squared_error(y_test, y_test_predicted))    
mae_test = mean_absolute_error(y_test, y_test_predicted)
mape_test = np.mean(np.abs((y_test - y_test_predicted) / y_test)) * 100
print("Test")
print(rms)
print(mae_test)
print(mape_test)

C:\Users\madhu\Anaconda3\lib\site-packages\ipykernel_launcher.py:2: DeprecationWarning: 
.ix is deprecated. Please use
.loc for label based indexing or
.iloc for positional indexing

See the documentation here:
http://pandas.pydata.org/pandas-docs/stable/indexing.html#deprecate_ix
  


Train
3.8378912751338086
2.96120873684
22.636220671502528
Test
3.8270784641254654
2.95821242522
22.651083702571988


In [179]:
#K-mean_6
X_train, X_test, y_train, y_test = train_test_split(kmean_6.ix[:, kmean_6.columns != 'int_rate'], 
                                            kmean_6['int_rate'], 
                                            test_size=0.30)
lm=linear_model.LinearRegression()
lm.fit(X_train,y_train)
y_train_predicted = lm.predict(X_train)
lm.fit(X_test,y_test)
y_test_predicted = lm.predict(X_test)
#finished square footage of the property, square footage of lot, tax amounts, number of bedrooms, and number of units
#MAE, RMS, MAPE
rms = sqrt(mean_squared_error(y_train, y_train_predicted))    
mae_train = mean_absolute_error(y_train, y_train_predicted) 
mape_train = np.mean(np.abs((y_train - y_train_predicted) / y_train)) * 100
print("Train")
print(rms)
print(mae_train)
print(mape_train)
#finished square footage of the property, square footage of lot, tax amounts, number of bedrooms, and number of units
#MAE, RMS, MAPE
rms = sqrt(mean_squared_error(y_test, y_test_predicted))    
mae_test = mean_absolute_error(y_test, y_test_predicted)
mape_test = np.mean(np.abs((y_test - y_test_predicted) / y_test)) * 100
print("Test")
print(rms)
print(mae_test)
print(mape_test)

C:\Users\madhu\Anaconda3\lib\site-packages\ipykernel_launcher.py:2: DeprecationWarning: 
.ix is deprecated. Please use
.loc for label based indexing or
.iloc for positional indexing

See the documentation here:
http://pandas.pydata.org/pandas-docs/stable/indexing.html#deprecate_ix
  


Train
3.915916685473573
3.03222388865
21.024905792104725
Test
3.9359741678248295
3.03410556875
21.025738610046368


In [180]:
#K-mean_7
X_train, X_test, y_train, y_test = train_test_split(kmean_7.ix[:, kmean_7.columns != 'int_rate'], 
                                            kmean_7['int_rate'], 
                                            test_size=0.30)
lm=linear_model.LinearRegression()
lm.fit(X_train,y_train)
y_train_predicted = lm.predict(X_train)
lm.fit(X_test,y_test)
y_test_predicted = lm.predict(X_test)
#finished square footage of the property, square footage of lot, tax amounts, number of bedrooms, and number of units
#MAE, RMS, MAPE
rms = sqrt(mean_squared_error(y_train, y_train_predicted))    
mae_train = mean_absolute_error(y_train, y_train_predicted) 
mape_train = np.mean(np.abs((y_train - y_train_predicted) / y_train)) * 100
print("Train")
print(rms)
print(mae_train)
print(mape_train)
#finished square footage of the property, square footage of lot, tax amounts, number of bedrooms, and number of units
#MAE, RMS, MAPE
rms = sqrt(mean_squared_error(y_test, y_test_predicted))    
mae_test = mean_absolute_error(y_test, y_test_predicted)
mape_test = np.mean(np.abs((y_test - y_test_predicted) / y_test)) * 100
print("Test")
print(rms)
print(mae_test)
print(mape_test)

C:\Users\madhu\Anaconda3\lib\site-packages\ipykernel_launcher.py:2: DeprecationWarning: 
.ix is deprecated. Please use
.loc for label based indexing or
.iloc for positional indexing

See the documentation here:
http://pandas.pydata.org/pandas-docs/stable/indexing.html#deprecate_ix
  


Train
3.3738462757238192
2.58337512395
23.645571904354206
Test
3.3365243655343533
2.55891123853
23.504137876036406


The text in the document by SHREYANTH REDDY BEZAWADA is licensed under CC BY 3.0 https://creativecommons.org/licenses/by/3.0/us/

Copyright 2018 SHREYANTH REDDY BEZAWADA AND ADITYA MOHAN KUMAR

Permission is hereby granted, free of charge, to any person obtaining a copy of this software and associated documentation files (the "Software"), to deal in the Software without restriction, including without limitation the rights to use, copy, modify, merge, publish, distribute, sublicense, and/or sell copies of the Software, and to permit persons to whom the Software is furnished to do so, subject to the following conditions:

The above copyright notice and this permission notice shall be included in all copies or substantial portions of the Software.

THE SOFTWARE IS PROVIDED "AS IS", WITHOUT WARRANTY OF ANY KIND, EXPRESS OR IMPLIED, INCLUDING BUT NOT LIMITED TO THE WARRANTIES OF MERCHANTABILITY, FITNESS FOR A PARTICULAR PURPOSE AND NONINFRINGEMENT. IN NO EVENT SHALL THE AUTHORS OR COPYRIGHT HOLDERS BE LIABLE FOR ANY CLAIM, DAMAGES OR OTHER LIABILITY, WHETHER IN AN ACTION OF CONTRACT, TORT OR OTHERWISE, ARISING FROM, OUT OF OR IN CONNECTION WITH THE SOFTWARE OR THE USE OR OTHER DEALINGS IN THE SOFTWARE.